# scrapers

> This module contains multiple scrapers, mostly for NSE

In [ ]:
# | default_exp experiments

In [ ]:
import nbdev
import numpy as np


In [ ]:
base_path = nbdev.config.get_config().lib_path
processed_data_dir = base_path / "../Data/Bhavcopy/Processed/"

In [ ]:
# | hide

# nbdev.nbdev_export()